# import json and perturb

In [1]:
import json
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy import constants
import os, sys
from cmcrameri import cm

##plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import corner
plt.style.use("Solarize_Light2")
plt.rcParams.update({"axes.edgecolor": "black"})
plt.rcParams.update({"text.color": "black"})
plt.rcParams.update({"axes.labelcolor": "black"})
plt.rcParams.update({"xtick.color": "black"})
plt.rcParams.update({"ytick.color": "black"})
plt.rcParams.update({"font.family": "monospace"})

In [2]:
def obs_noise(true, unc):
    noisy_obs = scipy.stats.norm(loc=true, scale=unc).rvs()
    return noisy_obs

In [26]:
with open(f'powerpuff/powerpuff.json') as f:
    star = json.load(f)

calc_effective_T = star['calc_effective_T'][0]
calc_effective_T_err = star['calc_effective_T'][1]

luminosity = star['luminosity'][0]
luminosity_err = star['luminosity'][1]

star_feh = star['star_feh'][0]
star_feh_err = star['star_feh'][1]

for obs_idx in range(5):
    obs_dict = {}
    
    obs_calc_effective_T = obs_noise(calc_effective_T, calc_effective_T_err)
    obs_dict['calc_effective_T'] = [obs_calc_effective_T, calc_effective_T_err]

    obs_luminosity = obs_noise(luminosity, luminosity_err)
    obs_dict['luminosity'] = [obs_luminosity, luminosity_err]

    obs_star_feh = obs_noise(star_feh, star_feh_err)
    obs_dict['star_feh'] = [obs_star_feh, star_feh_err]

    with open(f'powerpuff/obs{obs_idx}/obs{obs_idx}.json', 'w') as fp:
        json.dump(obs_dict, fp)
    

In [7]:
def json_to_obs(star_name, obs_idx):
    
    with open(f'powerpuff/{star_name}/{star_name}_l0.json') as f:
        star_l0 = json.load(f)

    with open(f'powerpuff/{star_name}/{star_name}_l012.json') as f:
        star_l02 = json.load(f)
    
    ## generate dict
    with open(f'powerpuff/obs{obs_idx}/obs{obs_idx}.json') as f:
        obs_dict = json.load(f)
    

    star_l0['calc_effective_T'] = obs_dict['calc_effective_T']
    star_l02['calc_effective_T'] = obs_dict['calc_effective_T']

    star_l0['luminosity'] = obs_dict['luminosity']
    star_l02['luminosity'] = obs_dict['luminosity']    

    star_l0['star_feh'] = obs_dict['star_feh']
    star_l02['star_feh'] = obs_dict['star_feh']

    os.makedirs(f'powerpuff/{star_name}/obs{obs_idx}', exist_ok=True)
    
    with open(f'powerpuff/{star_name}/obs{obs_idx}/obs{obs_idx}_l0.json', 'w') as fp:
        json.dump(star_l0, fp)

    with open(f'powerpuff/{star_name}/obs{obs_idx}/obs{obs_idx}_l012.json', 'w') as fp:
        json.dump(star_l02, fp)

In [10]:
for obs_idx in range(5):
    json_to_obs('bubbles',obs_idx)